In [ ]:
# 爬虫作业

In [4]:
# 必要包导入
import re
import time
import numpy as np
import pandas as pd
# 用于启动浏览器驱动
from selenium import webdriver
# 过滤筛选
from selenium.webdriver.common.by import By
# 页面解析
from bs4 import BeautifulSoup as bs

若没装driver则在终端中运行如下代码进行安装

$ brew install chromedriver

In [5]:
# 设置搜索关键词
keyword = '疫情下'
# 帖子标题 发帖人 发帖时间 link 每个帖子的评论 评论时间
# 初始化记录项
results = dict(
    帖子标题=[],
    发帖人=[],
    发帖时间=[],
    link=[],
    评论=[],
    评论时间=[]
)

## 网站一 babykindom

### 网站配置

URL: https://www.baby-kingdom.com/search.php?mod=forum&searchid=1658033935&srchtxt={关键词}&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword={关键词}

In [6]:
host = 'https://www.baby-kingdom.com/'
url = 'search.php?mod=forum&searchid=1658033935&srchtxt=关键词&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=关键词'

In [9]:
# 初始化浏览器 并加载搜索关键词页
driver = webdriver.Chrome()
driver.get(f'{host}{url.replace("关键词", keyword)}')

In [ ]:
# 获取所有帖子链接
html = driver.page_source
soup = bs(html,'html.parser')
links = []
contents = soup.find_all('h3', {'class': 'xs3'})
next_page = soup.find('a', {'class': 'nxt'})
for content in contents:
    content_link = list(content.children)[0]['href']
    links.append(content_link)
while next_page:
    driver = webdriver.Chrome()
    try:
        driver.get(f"{host}{next_page['href']}")
    except Exception as e:
        print(f"{next_page['href']} 请求失败 正在重试")
        try:
            time.sleep(3)
            driver.get(f"{host}{next_page['href']}")
        except Exception as e:
            print(f"{next_page['href']} 请求失败 跳过请求")
            driver.close()
            time.sleep(3)
            continue
    html = driver.page_source
    soup = bs(html,'html.parser')
    next_page = soup.find('a', {'class': 'nxt'})
    for content in contents:
        content_link = list(content.children)[0]['href']
        links.append(content_link)
print(f'获取链接结束, 共计 { len(links)}')
for link in links:
    print(link)

In [ ]:
saved_links = links.copy()

In [ ]:
links = saved_links.copy()
len(links)

In [ ]:
print(len(links))
author = ''
while links:
    # 进入每个帖子 获取帖子信息
    link = links.pop(0)
    driver = webdriver.Chrome()
    try:
        driver.get(link)
    except Exception as e:
        print(f'加载失败{link}，正在重试')
        try:
            driver.get(link)
        except Exception:
            print(f'加载失败，跳过; 剩余链接: {len(links)}')
            driver.close()
            continue
    html = driver.page_source
    soup = bs(html,'html.parser')
    title = ''.join(soup.find('a', {'id': 'thread_subject'}).strings)
    if not author:
        author = ''.join(soup.find('div', {'class', 'authi'}).strings).strip()
    author_time = ''.join(list(list(list(soup.find_all('div', {'class', 'authi'}))[1].children)[3].children)[1].strings)
    print(title, author, author_time, link)
    content_list = soup.find_all('div', {'class': 't_fsz'})
    comment_texts = []
    comment_times = []
    for content in content_list:
        text = ''.join(content.find('span').stripped_strings)
        text = re.sub('\d\d-\d+-\d\d \d\d:\d\d', '', text)
        text = re.sub('上傳下載附件\(.*\)', '', text)
        text = re.sub('\s+發表於 ', '', text)
        text = re.sub('本帖最後由 .+ 於  編輯', '', text)
        comment_texts.append(text)
    comment_list = soup.find_all('div', {'class', 'authi'})
    for comment in comment_list:
        comment_time_string = ''.join(comment.stripped_strings)
        comment_time = re.findall('\d\d-\d+-\d\d \d\d:\d\d', comment_time_string)
        if not comment_time:
            continue
        comment_times.append(comment_time)
    
    # 保存数据
    for text, comment_time in zip(comment_texts, comment_times):
        results['帖子标题'].append(title)
        results['发帖人'].append(author)
        results['发帖时间'].append(author_time)
        results['link'].append(link)
        results['评论'].append(text)
        results['评论时间'].append(comment_time)
    
    
    has_next = soup.find('div', {'class', 'pagination-control pagination-nextpage'})
    if has_next:
        if has_next.find('a'):
            link = f"{host}{has_next.find('a')['href']}"
            links = [link] + links
        else:
            author = ''
    else:
        author = ''


In [ ]:
df = pd.DataFrame(results)
df.replace('',np.nan, inplace =True)
df.dropna(subset=['评论'], inplace =True)
df.to_excel(f'网站1结果_{keyword}.xlsx')

## 网站二 hk高登

### 网站配置

URL: https://m.hkgolden.com/search/T/{关键词}


In [ ]:
# 网站2配置
host_2 = 'https://m.hkgolden.com'
url_2 = '/search/T/'

帖子标题 发帖人 发帖时间 link 每个帖子的评论 评论时间

In [ ]:
# 加载网站2入口
driver = webdriver.Chrome()
driver.get(host_2+url_2+keyword)

In [ ]:
html_2 = driver.page_source
soup2 = bs(html_2,'html.parser')
end_tag = soup2.find('div', {'class': 'jss100'})
while not end_tag:
    driver.execute_script('window.scrollBy(0, 10000)')
    time.sleep(0.5)
    html_2 = driver.page_source
    soup2 = bs(html_2,'html.parser')
    end_tag = soup2.find('div', {'class': 'jss100'})

In [ ]:
end_tag = soup2.find('div', {'class': 'jss100'})
print(end_tag)

In [ ]:
# 获取网站2需加载的连接
web_2_themes = []
next_links = soup2.find_all('a', {'href': re.compile("^/thread.*")})
for next_link in next_links:
    link = host_2+next_link['href']
    title = ''.join(list(list(list(next_link.parent.children)[1].children)[1].children)[0].strings)
    post_time = list(list(list(list(next_link.parent.children)[1].children)[0].children)[0].children)[1]['title']
    author = ''.join(list(list(list(list(next_link.parent.children)[1].children)[0].children)[0].children)[0].strings)
    theme = (author, title, post_time, link)
    print(theme)
    web_2_themes.append(theme)
print(f'获取全部帖子链接完成，共计{len(web_2_themes)}个')

In [ ]:
# 对每个帖子进行加载
page = 1
for author, title, author_time, link in web_2_themes:
    # 加载所有楼层
    url_content = link+f'/page/{page}'
    driver2 = webdriver.Chrome()
    print(url_content)
    try:
        driver2.get(url_content)
    except Exception as e:
        continue
    content = driver2.page_source
    soup_content = bs(content,'html.parser')
    texts = soup_content.find_all('div', {'data-role':'reply'})
    while texts:
        for text in texts:
            replay_text = ''.join(list(text.children)[1].strings)
            follow_time = list(list(list(list(list(text.children)[0].children)[0].children)[1].children)[1].children)[2]['title']
            print(replay_text, follow_time)
            # 保存数据
            results['帖子标题'].append(title)
            results['发帖人'].append(author)
            results['发帖时间'].append(author_time)
            results['link'].append(link)
            results['评论'].append(replay_text)
            results['评论时间'].append(follow_time)
        time.sleep(5)
        print()
        page += 1
        url_content = link+f'/page/{page}'
        print(url_content)
        try:
            driver2.get(url_content)
            content = driver2.page_source
            soup_content = bs(content,'html.parser')
            texts = soup_content.find_all('div', {'data-role':'reply'})
        except Exception as e:
            break
    page = 1
# driver2.close()

In [ ]:
results

In [ ]:
df = pd.DataFrame(results)
df.replace('',np.nan, inplace =True)
df.dropna(subset='评论', inplace =True)
df.to_excel(f'网站2结果_{keyword}.xlsx')

## 网站三 uwants

### 网站配置

URL: 

In [ ]:
driver.close()

In [ ]:
results

In [ ]:
df = pd.DataFrame(results)
df.replace('',np.nan, inplace =True)
df.dropna(subset=['评论'], inplace =True)
df.to_excel(f'网站2结果_{keyword}.xlsx')

## 网站三 uwants